In [148]:
# Импортируем другие библиотеки, а также их составляющие
import pandas as pd
import numpy as np
import datetime
import random
import statistics
import seaborn as sns


from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [149]:
# dataset_for_training - датасет, на котором наша модель будет обучаться
dataset_for_training = pd.read_csv('dataset.csv', sep = ',')
# dataset_for_predicting - датасет, на котором наша модель будет делать предсказания
dataset_for_predicting = pd.read_csv('dataset.csv', sep = ',')
# submission_example - датасет, из которого мы узнаем для каких людей нужно сделать предсказание
submission_example = pd.read_csv('submission_example.csv', sep = ',')
# SIZE - размер массивов с информацией о каждом партнёре. Размер выбран более 333515, чтобы мы могли обратиться в него по любому возможному pin-у
SIZE = 333515 + 420
# ALL_PEOPLE - pin-ы всех людей
ALL_PEOPLE = set(dataset_for_training.clientbankpartner_pin.values)

------------------
Создадим датасет для обучение

In [150]:
# SCORE - массив, в котором хранится вероятность оттока для каждого из партнёров
SCORE = [1] * SIZE
# CNT - массив, в котором хранится кол-во приведённых клиентов для каждого из партнёров
CNT = [0] * SIZE
# OFFLINE - массив, в котором хранится кол-во дней, прошедших с момента последного приглашения и до последнего для замера данных, для каждого из партнёров
OFFLINE = [100000000000000000] * SIZE
# CLIENTS - массив, в котором для каждого партнёра хранятся pin-ы клиентов, которых он пригласил
CLIENTS = [[] for i in range(SIZE)]
# DATES - массив, в котором хранятся все даты приглашения новых клиентов для каждого из партнёров
DATES = [[] for i in range(SIZE)]
# AVERAGE - массив, в котором хранится среднее кол-во дней между приглашениями новых клиентов для каждого из партнёров
AVERAGE = [0] * SIZE
# MEDIAN - массив, в котором хранится медианное значение кол-ва дней между приглашениями новых клиентов для каждого из партнёров
MEDIAN = [0] * SIZE
# DISPERSION - массив, в котором хранится дисперсия значений кол-ва дней между приглашениями новых клиентов для каждого из партнёров
DISPERSION = [0] * SIZE
# PARTNER_START - массив, в котором хранится дата начала сотрудничества с Alfа банком для каждого из партнёров
PARTNER_START = [''] * SIZE
# PARTNER_TIME - массив, в котором хранится кол-во дней, которые человек сотрудничает с Alfa банком, для каждого из партнёров
PARTNER_TIME = [0] * SIZE

In [151]:
# to_del - массив индексов строк, в которых содержатся дубликаты вида (x1, y1), (x2, y2), x1==x2 and y1==y2
to_del = []
for i in range(dataset_for_training.shape[0]): # Проходимся по всем строкам датасета для обучения
  # partner - pin текущего партнёра
  partner = dataset_for_training.clientbankpartner_pin[i]
  # client - pin клиента, которого пригласил текущий партнёр
  client = dataset_for_training.client_pin[i]
  if client in CLIENTS[partner]: # Если партнёр уже приглашал этого клиента, то нужно удалить текущую строчку
    to_del.append(i)
  else: # В противном случае запоминаем, что наш партнёр пригласил клиента с pin-ом (client)
    CLIENTS[partner].append(client)
for idx in to_del: # Проходимся по всем индексам ненужных строк и удаляем их
  dataset_for_training = dataset_for_training.drop(idx, axis=0)
# Возвращаем индексы к корректным значениям
dataset_for_training = dataset_for_training.reset_index()
dataset_for_training = dataset_for_training.drop(['index'], axis=1)

In [152]:
for i in range(dataset_for_training.shape[0]): # Проходимся по всем строкам датасета для обучения
  # partner - pin текущего партнёра
  partner = dataset_for_training.clientbankpartner_pin[i]
  # client - pin клиента, которого пригласил текущий партнёр
  client = dataset_for_training.client_pin[i]
  # client_date - дата приглашения партнёром клиента
  client_date = dataset_for_training.client_start_date[i]
  if "2020-06-01" <= client_date < "2020-09-01": # Если партнёр пригласил клиента в следующие 3 месяца после окончания замера данных, то указываем, что он не "оттёк"
    SCORE[partner] = 0
  elif client_date < "2020-06-01": # Если партнёр пригласил клиента до конца замера данных, то сохраняем эти данные о нём
    DATES[partner].append(client_date) # Добавляем текущекму партнёру дату приглашения клиента
    CNT[partner] += 1 # Увеличиваем число приглашённых клиентов нашим партнёром
    # Пробуем обновить значение кол-ва дней "бездействия" нашего партнёра
    OFFLINE[partner] = min(OFFLINE[partner], (datetime.datetime.strptime('2020-06-01', '%Y-%m-%d') - datetime.datetime.strptime(str(client_date), '%Y-%m-%d')).days)

In [153]:
for i in range(dataset_for_training.shape[0]): # Проходимся по всем строкам датасета для обучения
  # partner - pin текущего партнёра
  partner = dataset_for_training.clientbankpartner_pin[i]
  # partner_time - дата начала сотрудничества текущего партнёра с Alfa банком
  partner_time = dataset_for_training.partnerrolestart_date[i]
  PARTNER_START[partner] = partner_time # Устанавливаем дату сотрудничества текущего партнёра с Alfa банком
  # Устанавливаем значение кол-ва дней, которые текущий партнёр сотрудничает с Alfa банком
  PARTNER_TIME[partner] = (datetime.datetime.strptime('2020-06-01', '%Y-%m-%d') - datetime.datetime.strptime(str(partner_time), '%Y-%m-%d')).days

In [154]:
# Удаляем из датасета для обучения больше ненужные колонки
dataset_for_training = dataset_for_training.drop(['client_pin', 'client_start_date', 'partnerrolestart_date'], axis=1)
# Удаляем дубликаты
dataset_for_training = dataset_for_training.drop_duplicates(subset='clientbankpartner_pin', inplace=False)
# Приводим индексы строк к корректному виду
dataset_for_training = dataset_for_training.reset_index()
dataset_for_training = dataset_for_training.drop(['index'], axis=1)

In [155]:
for partner in ALL_PEOPLE: # Проходимся по pin-ам всех партнёров
  # DIFFS - массив дней в промежутках между приглашениями новых клиентов текущим партнёром, + в промежутке [начало сотрудничества; первое приглашение] + в промежутке [последние приглашение; конец замера данных]
  DIFFS = []
  # Располагаем значения кол-ва дней в промежутках в порядке возрастания
  DATES[partner] = sorted(DATES[partner])
  # Пропускаем партнёров, которые никого не пригласили, так как подобное невозможно в исходных данных => на подобных людях нет смысла обучать модель
  if CNT[partner] == 0:
    continue
  # Добавляем кол-во дней в промежутке [начало сотрудничества; первое приглашение]
  DIFFS.append((datetime.datetime.strptime(DATES[partner][0], '%Y-%m-%d') - datetime.datetime.strptime(PARTNER_START[partner], '%Y-%m-%d')).days)
  # Добавляем кол-во дней в промежутке [последние приглашение; конец замера данных]
  DIFFS.append((datetime.datetime.strptime('2020-06-01', '%Y-%m-%d') - datetime.datetime.strptime(DATES[partner][0], '%Y-%m-%d')).days)
  for i in range(1, len(DATES[partner])): # Проходимся по датам приглашения новых клиентов и добавляем значения кол-ва дней между ближайщими приглашениями
    DIFFS.append((datetime.datetime.strptime(str(DATES[partner][i]), '%Y-%m-%d') - datetime.datetime.strptime(str(DATES[partner][i - 1]), '%Y-%m-%d')).days)
  AVERAGE[partner] = sum(DIFFS) / len(DIFFS) # Считаем среднее значение по кол-ву дней между приглашениями
  MEDIAN[partner] = statistics.median(DIFFS) # Считаем медианное значение по кол-ву дней между приглашениями
  DISPERSION[partner] = statistics.variance(DIFFS, AVERAGE[partner]) # Считаем дисперсию по кол-ву дней между приглашениями

In [156]:
dataset_for_training

,clientbankpartner_pin,partner_src_type_ccode
0,122027,4
1,270277,4
2,238679,4
3,118398,4
4,10402,1
...,...,...
9466,315972,4
9467,4177,5
9468,326846,4
9469,4478,5


In [157]:
# Создаём новые нужные нам колонки в датасете
# cnt - кол-во приглашённых клиентов
dataset_for_training['cnt'] = [0] * dataset_for_training.shape[0]
# score - вероятность оттока клиента
dataset_for_training['score'] = [1] * dataset_for_training.shape[0]
# average - среднее кол-во дней между приглашениями
dataset_for_training['average'] = [0] * dataset_for_training.shape[0]
# med - медианное кол-во дней между приглашениями
dataset_for_training['med'] = [0] * dataset_for_training.shape[0]
# disp - дисперсия кол-ва дней между приглашениями
dataset_for_training['disp'] = [0] * dataset_for_training.shape[0]
# offline - время (кол-во дней), прошедшее с последнего приглашения и до конца замера данных
dataset_for_training['offline'] = [0] * dataset_for_training.shape[0]
# partner_period - время (кол-во дней), которое человек является партнёром Alfa банка
dataset_for_training['partner_period'] = [0] * dataset_for_training.shape[0]

In [ ]:
pd.options.mode.chained_assignment = None # Подавляем предупреждения
for i in range(dataset_for_training.shape[0]): # Проходимся по всем строкам в датасете для обучения
  # Присваиваем в строчках значения, соответствующие текущему партнёру с pin-ом clientbankpartner_pin[i]
  dataset_for_training.offline[i] = OFFLINE[dataset_for_training.clientbankpartner_pin[i]]
  dataset_for_training.average[i] = AVERAGE[dataset_for_training.clientbankpartner_pin[i]]
  dataset_for_training.cnt[i] = CNT[dataset_for_training.clientbankpartner_pin[i]]
  dataset_for_training.score[i] = SCORE[dataset_for_training.clientbankpartner_pin[i]]
  dataset_for_training.partner_period[i] = PARTNER_TIME[dataset_for_training.clientbankpartner_pin[i]]
  dataset_for_training.med[i] = MEDIAN[dataset_for_training.clientbankpartner_pin[i]]
  dataset_for_training.disp[i] = DISPERSION[dataset_for_training.clientbankpartner_pin[i]]

In [ ]:
dataset_for_training = dataset_for_training.sort_index(axis=1) # Сортируем колонки по названию
dataset_for_training = dataset_for_training[dataset_for_training.cnt > 0] # Оставляем только строчки, в которых партнёры пригласили хотя бы 1-го клиента
# Приводим индексы к корректному формату
dataset_for_training = dataset_for_training.reset_index()
dataset_for_training = dataset_for_training.drop(['index'], axis=1)

In [ ]:
# Создаем из нашего датасета для обучения csv файл
dataset_for_training.to_csv('training.csv', index=False)

Датасет для обучения создан

---------------------

In [ ]:
# CNT_INVITES - массив, в котором хранится кол-во приведённых клиентов для каждого из партнёров
CNT_INVITES = [0] * SIZE
# INVITES - массив, в котором для каждого партнёра хранятся pin-ы клиентов, которых он пригласил
INVITES = [[] for i in range(SIZE)]
# DATES_OF_INVITES - массив, в котором хранятся все даты приглашения новых клиентов для каждого из партнёров
DATES_OF_INVITES = [[] for i in range(SIZE)]
# OFFLINE - массив, в котором хранится кол-во дней, прошедших с момента последного приглашения и до последнего для замера данных, для каждого из партнёров
OFFLINE = [10000000000000000000000] * SIZE
HAS_WORKED = [0] * SIZE
# AVERAGE - массив, в котором хранится среднее кол-во дней между приглашениями новых клиентов для каждого из партнёров
AVERAGE = [0] * SIZE
# PARTNER_TIME - массив, в котором хранится кол-во дней, которые человек сотрудничает с Alfa банком, для каждого из партнёров
PARTNER_TIME = [0] * SIZE
# PARTNER_START - массив, в котором хранится дата начала сотрудничества с Alfа банком для каждого из партнёров
PARTNER_START = [''] * SIZE
# MEDIAN - массив, в котором хранится медианное значение кол-ва дней между приглашениями новых клиентов для каждого из партнёров
MEDIAN = [0] * SIZE
# DISPERSION - массив, в котором хранится дисперсия значений кол-ва дней между приглашениями новых клиентов для каждого из партнёров
DISPERSION = [0] * SIZE

In [ ]:
# to_del - массив индексов строк, в которых содержатся дубликаты вида (x1, y1), (x2, y2), x1==x2 and y1==y2
to_del = [] # Удаляем строки с дубликатами
for i in range(dataset_for_predicting.shape[0]): # Проходимся по всем строкам датасета для предсказания
  # partner_pin - pin текущего партнёра
  partner_pin = dataset_for_predicting.clientbankpartner_pin[i]
  # client_pin - pin клиента, которого пригласил текущий партнёр
  client_pin = dataset_for_predicting.client_pin[i]
  if client_pin in INVITES[partner_pin]: # Если партнёр уже приглашал этого клиента, то нужно удалить текущую строчку
    to_del.append(i)
  else: # В противном случае запоминаем, что наш партнёр пригласил клиента с pin-ом (client_pin)
    INVITES[partner_pin].append(client_pin)
for idx in to_del: # Проходимся по всем индексам ненужных строк и удаляем их
  dataset_for_predicting = dataset_for_predicting.drop(idx, axis=0)
# Возвращаем индексы к корректным значениям
dataset_for_predicting = dataset_for_predicting.reset_index()
dataset_for_predicting = dataset_for_predicting.drop(["index"], axis=1)

In [ ]:
for i in range(dataset_for_predicting.shape[0]):
  # partner_pin - pin текущего партнёра
  partner_pin = dataset_for_predicting.clientbankpartner_pin[i]
  # client_pin - pin клиента, которого пригласил текущий партнёр
  client_pin = dataset_for_predicting.client_pin[i]
  # date_of_becoming_a_partner - дата становления партнёром
  date_of_becoming_a_partner = dataset_for_predicting.partnerrolestart_date[i]
  # date_of_inviting - дата приглашения партнёром клиента
  date_of_inviting = dataset_for_predicting.client_start_date[i]
  CNT_INVITES[partner_pin] += 1 # Увеличиваем число приглашённых клиентов нашим партнёром
  DATES_OF_INVITES[partner_pin].append(date_of_inviting) # Добавляем текущекму партнёру дату приглашения клиента
  # Пробуем обновить значение кол-ва дней "бездействия" нашего партнёра
  OFFLINE[partner_pin] = min(OFFLINE[partner_pin], (datetime.datetime.strptime('2020-12-01', '%Y-%m-%d') - datetime.datetime.strptime(str(date_of_inviting), '%Y-%m-%d')).days)
  # Устанавливаем значение кол-ва дней, которые текущий партнёр сотрудничает с Alfa банком
  HAS_WORKED[partner_pin] = (datetime.datetime.strptime('2020-12-01', '%Y-%m-%d') - datetime.datetime.strptime(str(date_of_becoming_a_partner), '%Y-%m-%d')).days

In [ ]:
for i in range(dataset_for_predicting.shape[0]):
  # partner - pin текущего партнёра
  partner = dataset_for_predicting.clientbankpartner_pin[i]
  # partner_time - дата начала сотрудничества текущего партнёра с Alfa банком
  partner_time = dataset_for_predicting.partnerrolestart_date[i]
  PARTNER_START[partner] = partner_time # Устанавливаем дату сотрудничества текущего партнёра с Alfa банком
  # Устанавливаем значение кол-ва дней, которые текущий партнёр сотрудничает с Alfa банком
  PARTNER_TIME[partner] = (datetime.datetime.strptime('2020-12-01', '%Y-%m-%d') - datetime.datetime.strptime(str(partner_time), '%Y-%m-%d')).days

In [ ]:
# Удаляем из датасета для предсказания больше ненужные колонки
dataset_for_predicting = dataset_for_predicting.drop(['client_start_date', 'partnerrolestart_date', 'client_pin'], axis=1)
# Удаляем дубликаты
dataset_for_predicting = dataset_for_predicting.drop_duplicates(subset='clientbankpartner_pin', inplace=False)
# Приводим индексы строк к корректному виду
dataset_for_predicting = dataset_for_predicting.reset_index()
dataset_for_predicting = dataset_for_predicting.drop(['index'], axis=1)

In [ ]:
for partner in ALL_PEOPLE: # Проходимся по pin-ам всех партнёров
  # DIFFS - массив дней в промежутках между приглашениями новых клиентов текущим партнёром, + в промежутке [начало сотрудничества; первое приглашение] + в промежутке [последние приглашение; конец замера данных]
  DIFFS = []
  # Располагаем значения кол-ва дней в промежутках в порядке возрастания
  DATES_OF_INVITES[partner] = sorted(DATES_OF_INVITES[partner])
  # Добавляем кол-во дней в промежутке [начало сотрудничества; первое приглашение]
  DIFFS.append((datetime.datetime.strptime(DATES_OF_INVITES[partner][0], '%Y-%m-%d') - datetime.datetime.strptime(PARTNER_START[partner], '%Y-%m-%d')).days)
  # Добавляем кол-во дней в промежутке [последние приглашение; конец замера данных]
  DIFFS.append((datetime.datetime.strptime('2020-12-01', '%Y-%m-%d') - datetime.datetime.strptime(DATES_OF_INVITES[partner][0], '%Y-%m-%d')).days)
  for i in range(1, len(DATES_OF_INVITES[partner])): # Проходимся по датам приглашения новых клиентов и добавляем значения кол-ва дней между ближайщими приглашениями
    DIFFS.append((datetime.datetime.strptime(str(DATES_OF_INVITES[partner][i]), '%Y-%m-%d') - datetime.datetime.strptime(str(DATES_OF_INVITES[partner][i - 1]), '%Y-%m-%d')).days)
  AVERAGE[partner] = sum(DIFFS) / len(DIFFS) # Считаем среднее значение по кол-ву дней между приглашениями
  MEDIAN[partner] = statistics.median(DIFFS) # Считаем медианное значение по кол-ву дней между приглашениями
  DISPERSION[partner] = statistics.variance(DIFFS, AVERAGE[partner]) # Считаем дисперсию по кол-ву дней между приглашениями

In [ ]:
# Приводим
# dataset_for_predicting.partner_src_type_ccode = dataset_for_predicting.partner_src_type_ccode.astype(str)

In [ ]:
# Создаём новые нужные нам колонки в датасете

# offline - время (кол-во дней), прошедшее с последнего приглашения и до конца замера данных
dataset_for_predicting['offline'] = [0] * dataset_for_predicting.shape[0]
# average - среднее кол-во дней между приглашениями
dataset_for_predicting['average'] = [0] * dataset_for_predicting.shape[0]
# cnt - кол-во приглашённых клиентов
dataset_for_predicting['cnt'] = [0] * dataset_for_predicting.shape[0]
# partner_period - время (кол-во дней), которое человек является партнёром Alfa банка
dataset_for_predicting['partner_period'] = [0] * dataset_for_predicting.shape[0]
# med - медианное кол-во дней между приглашениями
dataset_for_predicting['med'] = [0] * dataset_for_predicting.shape[0]
# disp - дисперсия кол-ва дней между приглашениями
dataset_for_predicting['disp'] = [0] * dataset_for_predicting.shape[0]

In [ ]:
for i in range(dataset_for_predicting.shape[0]): # Проходимся по всем строкам в датасете для предсказания
  # Присваиваем в строчках значения, соответствующие текущему партнёру с pin-ом clientbankpartner_pin[i]
  dataset_for_predicting.offline[i] = OFFLINE[dataset_for_predicting.clientbankpartner_pin[i]]
  dataset_for_predicting.average[i] = AVERAGE[dataset_for_predicting.clientbankpartner_pin[i]]
  dataset_for_predicting.cnt[i] = CNT_INVITES[dataset_for_predicting.clientbankpartner_pin[i]]
  dataset_for_predicting.partner_period[i] = PARTNER_TIME[dataset_for_predicting.clientbankpartner_pin[i]]
  dataset_for_predicting.med[i] = MEDIAN[dataset_for_predicting.clientbankpartner_pin[i]]
  dataset_for_predicting.disp[i] = DISPERSION[dataset_for_predicting.clientbankpartner_pin[i]]

In [ ]:
# idx_to_keep - массив с индексами строк, которые нам нужно оставить. Оставить нам нужно только те строки, в которых партнёром является человеком для которого намн нужно предсказать вероятность оттока
idx_to_keep = []
for i in range(dataset_for_predicting.shape[0]): # Проходимся по по всем строкам датасета для предсказаний
  # partner - pin текущего партнёра
  partner = dataset_for_predicting.clientbankpartner_pin[i]
  if partner in submission_example.clientbankpartner_pin.values: # Если pin нашего партнёра встречается в submission_example, то для нашего партнёра нужно будет сделать предсказание
    idx_to_keep.append(i)
# Оставляем только нужные нам строки
dataset_for_predicting = dataset_for_predicting.iloc[idx_to_keep]
# Приводим индексы к корректному формату
dataset_for_predicting = dataset_for_predicting.reset_index()
dataset_for_predicting = dataset_for_predicting.drop(['index'], axis=1)

In [ ]:
# Сортируем колонки датасета для предсказания по названию
dataset_for_predicting = dataset_for_predicting.sort_index(axis=1)

In [ ]:
# Считываем датасет для обучения модели
DATA_TRAIN = pd.read_csv('training.csv', sep = ',')

In [ ]:
# Сортируем колонки датасета для обучения по названию
DATA_TRAIN = DATA_TRAIN.sort_index(axis=1)

In [ ]:
# X_pred - датасет для опредсказания без pin-ов партнёров
X_pred = dataset_for_predicting.drop(['clientbankpartner_pin'], axis=1)
# y_pred - массив значений вероятности оттока партнёров, для которых нам нужно сделать предсказание
y_pred = [0] * dataset_for_predicting.shape[0]
# RANDOM_SEEDS_1, RANDOM_SEEDS_2, RANDOM_SEEDS_3 - массивы случайных, заранее созданных сидов для тренировки модели
RANDOM_SEEDS_1 = [3891800075, 3016415865, 2176061081, 1731419478, 3783895874, 2764739521, 2782060052, 3524179692, 2844149057, 1244701250, 3899862384, 2813020269, 1510783940, 257708917, 2500441839, 128024146, 4035610767, 3541990627, 175914092, 3535216303]
RANDOM_SEEDS_2 = [1266929725, 3214219494, 2720626381, 2266809387, 482891602, 978069738, 1175652263, 1309829645, 3403918683, 182045614, 627045617, 1344754375, 813630871, 923267019, 2773042947, 296316750, 3706821871, 1672336920, 2655620765, 2699758462]
RANDOM_SEEDS_3 = [1531906963, 262295212, 1782349066, 1500120831, 1676857053, 601432304, 2432272467, 3151521108, 3460296720, 43961157, 899070798, 4100257218, 4169689651, 406556915, 3431974749, 363353800, 3920168246, 1250644752, 1230826994, 348884761]
for i in range(len(RANDOM_SEEDS_1)): # Проходимся по индексам массивов с сидами
  DATA_TRAIN = shuffle(DATA_TRAIN, random_state=RANDOM_SEEDS_1[i]) # Перемешиваем датасет для обучения, используя сид из 1-го массива

  # y - таргет
  y = DATA_TRAIN.score
  # X - датасет для обучения без указанного таргета
  X = DATA_TRAIN.drop(['clientbankpartner_pin', 'score'], axis=1)

  # Разбиваем датасет для обучения на датасеты, на которых модель будет тренироваться и на которых будет проверяться на точность
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEEDS_2[i])

  # Создаём модель
  model = CatBoostClassifier(loss_function="CrossEntropy", n_estimators = 5000, eval_metric='AUC', learning_rate=0.008, random_seed=RANDOM_SEEDS_3[i], bootstrap_type="Bernoulli", cat_features=['partner_src_type_ccode'])
  # Обучаем её
  model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100, cat_features=['partner_src_type_ccode'], early_stopping_rounds=400)
  # Сохраняем предсказания того, что партнёр "оттечёт"
  CatBoost_y_pred = model.predict_proba(X_pred)[:, 1]
  # Добавляем эти вероятности в y_pred для дальнейшего усреднения
  for j in range(len(CatBoost_y_pred)):
    y_pred[j] += CatBoost_y_pred[j]
# Усредняем наши предсказанные вероятности
for i in range(len(y_pred)):
  y_pred[i] /= len(RANDOM_SEEDS_1)

In [ ]:
# FROM_PARTNER_TO_SCORE - массив, который для каждого pin-а партнёра хранит вероятность его оттока
FROM_PARTNER_TO_SCORE = [0] * SIZE
for i in range(len(y_pred)): # Проходимся по всем предсказаниям и сопоставляем партнёру его вероятность оттока
  FROM_PARTNER_TO_SCORE[dataset_for_predicting.clientbankpartner_pin[i]] = y_pred[i]

In [ ]:
# ouput - наш ответ
output = submission_example[:] # Делаем копию submission_example, чтобы записать в пропусках вероятности
for i in range(output.shape[0]): # Проходимся по всем строкам ответа и указываем партнёру его вероятность оттока
  output.score[i] = FROM_PARTNER_TO_SCORE[output.clientbankpartner_pin[i]]
# формируем csv файл из output
output.to_csv('output.csv', index=False)